In [ ]:
# Install all the required libraries for Speech-to-text and speaker diarization
%%capture
!git clone https://github.com/promptslab/Promptify.git
!pip3 install openai
!pip install tiktoken
!pip install openai -q
!pip install huggingface_hub
!pip install bert_score
!pip install anthropic
!pip install cohere
!pip install --upgrade PyPDF2 python-docx

nltk.download('punkt')


In [ ]:
# Import required libraries
%cd /content/Promptify
import numpy as np
from bert_score import BERTScorer, score
from transformers import DebertaTokenizer, DebertaModel
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
from promptify import OpenAI, Prompter
from pprint import pprint
from IPython.display import Markdown, display
from IPython.core.display import display, HTML
import time
import openai
import os
import re
import ast
import nltk
import json
import glob
from docx import Document
from PyPDF2 import PdfReader
from google.colab import drive


In [ ]:
#OpenAI API key. [PN: I have removed my token purposefully due to privacy restrictions from Grantify Limited]
api_key  = "sk-"

# Create an instance for the OpenAI model, Currently supporting Openai's all model, In future adding more generative models from Hugginface and other platforms
model = OpenAI(api_key)
nlp_prompter = Prompter(model)

# Mount the path where the audio files(input files) are stored
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Function to extract entities using chapt GPT 4
def get_entities_question_guide_openai(question, guideline, MODEL="gpt-4"):
    time.sleep(1)

    if not question or not guideline:
      return ""

    system_role = "You are an helpful agent that performs named entity recognition (NER) on the given text and show a list of comma separated entity types. Output empty string of no entities are present."
    user_role = f"Given the QUESTION and GUIDLINES, predict different named entities that are present in the text. Only show the complete named entity type in comma separated way:\n"
    input_text = f"QUESTION: {question} \nGUIDELINES: {guideline}"
    completions = openai.ChatCompletion.create(
                model = MODEL,
                messages = [
                    {'role': 'system', 'content': system_role},
                    {'role': 'user', 'content': user_role + input_text},
                ],
                temperature = 0
                )
    output = completions['choices'][0]['message']['content']
    return output

In [ ]:
# Function to execute the prompt as input using OpenAI chapt GPT 4
def openai_chat_completion_response(final_prompt):
  response = openai.ChatCompletion.create(
              model="gpt-4",
              messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": USER_PROMPT_1},
                    {"role": "assistant", "content": ASSISTANT_PROMPT_1},
                    {"role": "user", "content": final_prompt}
                ]
            )

  return response['choices'][0]['message']['content'].strip(" \n")

In [ ]:
# Code block to convert docx of pdf files to txt format
def convert_pdf_to_txt(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf = PdfReader(file)
        text = ''
        for page in range(len(pdf.pages)):
            text += pdf.pages[page].extract_text().strip() + '\n'
        return text

def convert_word_to_txt(word_path):
    doc = Document(word_path)
    text = ''
    for paragraph in doc.paragraphs:
        text += paragraph.text + '\n'
    return text

def convert_documents_to_txt(folder_path):
    files = glob.glob(os.path.join(folder_path, '*'))
    for file_path in files:
      print('0')
      if file_path.endswith('.pdf'):
          print('1')
          text = convert_pdf_to_txt(file_path)
          txt_path = os.path.splitext(file_path)[0] + '.txt'
          with open(txt_path, 'w', encoding='utf-8') as txt_file:
              txt_file.write(text)
          print(f'Converted {file_path} to {txt_path}')
      elif file_path.endswith('.docx'):
          text = convert_word_to_txt(file_path)
          txt_path = os.path.splitext(file_path)[0] + '.txt'
          with open(txt_path, 'w', encoding='utf-8') as txt_file:
              txt_file.write(text)
          print(f'Converted {file_path} to {txt_path}')
      else:
          print(f'Skipped {file_path} (unsupported file format)')


# Provide the folder path containing PDF and Word documents
folder_path = "/content/drive/MyDrive/Audio_Transcripts/"
print('Folder Path:',folder_path)
convert_documents_to_txt(folder_path)
print('Done')


In [ ]:
filename = "/content/drive/MyDrive/Audio_Transcripts/GMT20230222-115812_Recording.transcript_txt.txt"

In [ ]:
# Code for entity extraction
def read_large_file(filename, chunk_size):
    with open(filename, 'rb') as file:
        chunk = ''
        while True:
            # Read chunk_size number of characters from the file
            data = file.read(chunk_size)
            # Convert the content to a string
            data = data.decode('utf-8')

            # Break the loop if the end of the file is reached
            if not data:
                break

            # Append the current chunk to the overall chunk
            chunk += data

            # Split the chunk into separate tokens (e.g., words)
            tokens = chunk.split()

            # Check if the last token is complete or partial
            last_token = tokens[-1]
            last_token_length = len(last_token)

            # If the last token is complete, yield the chunk and reset it
            if last_token_length == chunk_size:
                yield chunk
                chunk = ''

            # If the last token is partial, remove it from the chunk
            elif last_token_length < chunk_size:
                chunk = ' '.join(tokens[:-1]) + ' '

        # Yield the remaining chunk (if any)
        if chunk:
            yield chunk

# Example usage

chunk_size = 8000

for chunk in read_large_file(filename, chunk_size):
    print('CHUNKKKKKKKK',len(chunk))
    # Split the chunk into smaller subchunks of maximum allowed length
    subchunks = [chunk[i:i+8192] for i in range(0, len(chunk), 8192)]
    print('subchunk:',len(subchunks))
    k=0
    for subchunk in subchunks:
        print("k",k)
        k=k+1
        print("subchunk:",subchunk)
        # Do something with each subchunk (e.g., process it)

        SYSTEM_PROMPT = "You are a smart and intelligent Named Entity Recognition (NER) system. I will provide you the definition of the entities you need to extract and the sentence from which you need to extract the entities and the output in given format with examples, but do not use the examples as the actual output."

        USER_PROMPT_1 = "Are you clear about your role?"

        ASSISTANT_PROMPT_1 = "Sure, I'm ready to help you with your NER task. Please provide me with the necessary information to get started."

        PROMPT = (
            "Entity Definition:\n"
            "1. Company name: The official name by which a company is registered and known legally. It is the unique identifier for a company and is used in various official documents and communications like Google, samsung, Apple etc.\n"
            "2. Company incorporation date: This refers to the date when a company is officially formed and registered with the appropriate government authority. It marks the starting point of the company's legal existence. Dates can also be in natural language.\n"
            "3. Company registration number: It is a unique alphanumeric identifier assigned to a company by the government authority responsible for registering companies. This number is used for official purposes, such as taxation, legal filings, and identification of the company in various records.\n"
            "4. Unique Taxpayer Reference: It is a unique identifier assigned to an individual or company by the tax authority in the United Kingdom, Her Majesty's Revenue and Customs (HMRC). It is also called as UTR. The UTR is used to track and manage tax obligations, including R&D tax credits, for the entity associated with it. It is important to note that UTR specifically refers to the UK tax system and may have similar equivalents in other countries.\n"
            "5. Accounting period: It refers to the specific timeframe during which a company's eligible research and development expenses are assessed for the purpose of claiming tax credits. It is the period for which financial records and transactions are prepared and reported to determine the qualifying R&D activities and associated expenses.\n"
            "6. Client email: The email address associated with a client or customer involved in the R&D project for communication purposes.\n"
            "7. Confirm Ltd Company: Confirmation of whether the company is a limited company, which is a legal business structure with limited liability for its shareholders. The answer should be a yes or no.\n"
            "8. Confirm SME: Confirmation of whether the company meets the criteria of a small or medium-sized enterprise (SME) which typically includes having fewer than 500 employees and an annual turnover below £100 million. (Conditions: <500 employees, <£100m annual turnover). The answer should be a yes or no.\n"
            "9. Confirm no other government grants/subsidies in past 3 periods: Confirmation of whether the company or claimant has received any other government grants or subsidies during the previous three accounting periods. The answer should be a yes if the company or claimant received any grants or subsidies and a no if not received.\n"
            "10. Confirm no linked enterprises: Confirmation of whether the company has any linked enterprises, indicated by either owning more than 50% of another business or having another company own more than 50% of the company. The answer should be a yes or no.\n"
            "11. Confirm no partner enterprises: Confirmation of whether the company has any partner enterprises, indicated by either owning between 25% and 50% of another company or having another company own between 25% and 50% of the company. The answer should be a yes or no.\n"
            "12. Industry Classification: You are an industry classification expert assisting a client in determining the industry classification for their business. Your client provides you with a brief description of their business, and you are tasked with identifying and classifying the industry in which their business operates. Provide a detailed response explaining the industry classification and the factors that contribute to it."
            "13. Elevator Pitch: You are a marketing consultant working with a client who needs help crafting an effective elevator pitch for their business. The client provides you with a brief description of their business, and you are tasked with creating a compelling elevator pitch that succinctly conveys the unique value proposition and key benefits of their business. Write an elevator pitch that captures the essence of their business and highlights its competitive advantage, target audience, and main offerings in a concise and persuasive manner."
            "14. Name of CTO:You are an R&D tax credits consultant working with a company to help them navigate the process of claiming R&D tax credits. The company seeks assistance in providing the name of their Chief Technology Officer (CTO) for inclusion in the tax credit application. Write a response guiding them on how to identify and document the CTO's name within the specific context of R&D tax credits, considering their role and responsibilities in driving the company's technological and innovation strategies."
            "15. Experience and/or Qualifications: It refers to the expertise and qualifications of individuals involved in the company's research and development activities. These competent professionals play a crucial role in driving and overseeing the R&D projects that qualify for tax credits."
            "16. Team Member 1: You are an R&D tax credits consultant assisting a company in preparing their R&D tax credit claim. The company requires information about Team Member 1, including their name, role, and experience/qualifications."
            "17. Team Member 2: You are an R&D tax credits consultant assisting a company in preparing their R&D tax credit claim. The company requires information about Team Member 2, including their name, role, and experience/qualifications."
            "\n"
            "Output Format:\n"
            "{{'Company name': [list of entities present], 'Company incorporation date': [list of entities present], 'Company registration number': [list of entities present],'Unique Taxpayer Reference': [list of entities present], 'Accounting period': [list of entities present], 'Client email': [list of entities present], 'Confirm Ltd Company': [list of entities present], 'Confirm SME': [list of entities present], 'Confirm no other government grants/subsidies in past 3 periods': [list of entities present], 'Confirm no linked enterprises': [list of entities present], 'Confirm no partner enterprises': [list of entities present], 'Industry Classification': [list of entities present], 'Elevator Pitch': [list of entities present], 'Name of CTO': [list of entities present], 'Experience and/or Qualifications': [list of entities present], 'Team Member 1': [list of entities present], 'Team Member 2': [list of entities present]}}\n"
            "If no entities are presented in any categories keep it None\n"
            "\n"
            "Examples:\n"
            "\n"
            "1. Sentence: *(Here I have provide appropriate example of the Grantify's client to the OpenAI model. I have removed this example due to provacy restrictions from Grantify Limited.)*\n"

            "Output: {{'Company name': ['ABC Ltd'], 'Company incorporation date': ['31st March 2022'], 'Company registration number': ['XYZ'],'Unique Taxpayer Reference':['PQR'], 'Accounting period': ['31st March 2022'], 'Client email': ['None'],'Confirm Ltd Company': ['yes'],'Confirm SME': ['yes'], 'Confirm no other government grants/subsidies in past 3 periods': ['No'], 'Confirm no linked enterprises': [''], 'Confirm no partner enterprises': [''], 'Industry Classification': [ 'Renewable Energy', 'Hydropower Generation'], 'Elevator Pitch': ['At Kinetic Hydro Ltd, we're revolutionizing access to electricity for underserved communities worldwide. Our innovative river turbines harness the untapped energy resource of flowing water, providing continuous (baseload) power that contributes to the energy security of rural areas. With easy transportation, installation, and maintenance, our turbines are designed specifically for remote locations. By offering a cost-effective alternative to grid expansion, we address the pressing energy access needs in Sub Saharan Africa and South and East Asia. Join us in powering the future sustainably and making a meaningful impact on the lives of millions.'], 'Name of CTO': ['None'], 'Experience and/or Qualifications': ['None'], 'Team Member 1': ['Dr Richard Montague, Director and Engineer, over 17 years experience'], 'Team Member 2': ['Donald Naylor, Director and Engineer, over 20 years experience'], 'Team Member 3': ['Raafey Andrabi, Engineering Intern'] }}\n"
            "\n"

            "2. Sentence: {}\n"
            "Output: "
        )

        PROMPT = PROMPT.format(subchunk)
        ners = openai_chat_completion_response(PROMPT)
        print(ners)


In [ ]:
# Logic for text Summarization and semantic similarity analysis starts from here

In [ ]:
# function for getting DeBERTa similarity

def compute_similarity_deberta(sentence1, sentence2):
    # Tokenize input sentences
    encoded_input = tokenizer.encode_plus(sentence1, sentence2, padding='longest', truncation=True, return_tensors='pt')

    with torch.no_grad():
        # Obtain the DeBERTa embeddings for the input sentences
        outputs = model_deberta(**encoded_input)
        embeddings = outputs.last_hidden_state.squeeze(0)

    # Compute cosine similarity between the embeddings
    similarity_score = torch.nn.functional.cosine_similarity(embeddings[0], embeddings[1], dim=0).item()
    return similarity_score

def compute_similarity_matrix_deberta(candidate_list, reference_list):
    similarity_matrix = []
    for candidate in candidate_list:
        row = []
        for reference in reference_list:
            similarity_score = compute_similarity_deberta(candidate, reference)
            row.append(similarity_score)
        similarity_matrix.append(row)
    return similarity_matrix

# Load DeBERTa model and tokenizer
model_name_deberta = 'microsoft/deberta-base'
tokenizer = DebertaTokenizer.from_pretrained(model_name_deberta)
model_deberta = DebertaModel.from_pretrained(model_name_deberta)


In [ ]:
# function for getting RoBERTa similarity

def compute_similarity_roberta(sentence1, sentence2):
    encoded_input = tokenizer(sentence1, sentence2, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model_roberta(**encoded_input)
    logits = outputs.logits
    softmax = torch.softmax(logits, dim=1)
    similarity_score = softmax[0][1].item()
    return similarity_score

def compute_similarity_matrix_roberta(candidate_list, reference_list):
    similarity_matrix = []
    for candidate in candidate_list:
        row = []
        for reference in reference_list:
            similarity_score = compute_similarity_roberta(candidate, reference)
            row.append(similarity_score)
        similarity_matrix.append((candidate, reference, similarity_score))
    return similarity_matrix

model_name_roberta = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name_roberta)
model_roberta = RobertaForSequenceClassification.from_pretrained(model_name_roberta)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# BERTScore Implementation - Evaluation metric for Summarization
def generate_bert_score(cands, refs):
  sentences_cands = nltk.sent_tokenize(cands)

  # list_of_sentences_cands = [sentence.split() for sentence in sentences_cands]

  print(sentences_cands)

  sentences_refs = nltk.sent_tokenize(refs)

  # list_of_sentences_refs = [sentence.split() for sentence in sentences_refs]

  list_of_lists_refs = [sentences_refs.copy() for _ in range(len(sentences_refs))]

  print(list_of_lists_refs)

  # print(sentences_refs)

  scorer = BERTScorer(lang="en", rescale_with_baseline=True)
  P, R, F1 = scorer.score(sentences_cands, list_of_lists_refs)

  print("F1 Score:",F1)
  print("F1 Mean Score:",F1.mean())

  return F1, F1.mean()

In [ ]:
# function for generating the summary by applying the HMRC guidelines to the summary generated. This is used for answers 3, 4 and 5.
def apply_hmrc_guidelines(full_summary, hmrc_reference):
    prompt = f"Summary: {full_summary}\nReference: {hmrc_reference}\nQuestion: {question}\nApply HMRC guidelines to modify the summary according on the prompts mentioned in the question:"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1024,
        temperature=0.7,
        n=1,
        stop=None,
        timeout=10
    )

    modified_summary = response.choices[0].message.content.strip().split("Summary: ")[-1]
    return modified_summary


In [ ]:
# function for extracting text from the call transcription file i.e., the input file
def extract_text(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    return text

# Provide the path to your text file
file_path = filename

# Call the function to extract the text
extracted_text = extract_text(file_path)

# Print the extracted text
print(extracted_text)


In [ ]:
# extract the words by splitting at space as a delimiter
words = extracted_text.split(" ")

# show the first 20 words
words[:20]

In [ ]:
# create an array of 6 words as a chunk to be suitable for processing
chunks = np.array_split(words, 6)

chunks

In [ ]:
sentences = ' '.join(list(chunks[0]))

sentences

In [ ]:
# sample run
prompt = f"{sentences}\n\ntl;dr:"

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.3, # The temperature controls the randomness of the response, represented as a range from 0 to 1. A lower value of temperature means the API will respond with the first thing that the model sees; a higher value means the model evaluates possible responses that could fit into the context before spitting out the result.
    max_tokens=1024,
    top_p=1, # Top P controls how many random results the model should consider for completion, as suggested by the temperature dial, thus determining the scope of randomness. Top P’s range is from 0 to 1. A lower value limits creativity, while a higher value expands its horizons.
    frequency_penalty=0,
    presence_penalty=1
)

response_text=response.choices[0].message.content
response_text

In [ ]:
# Code block for text summariZation for question 1
summary_responses = []

question = """Prompt:

Please provide information regarding the introduction and background of the company, including the incorporation date, registration number, reference number, company background, innovative solution, and any research and development (R&D) tax relief.

Instructions:

Incorporation Date:
Please provide the date when the company was incorporated.
Registration Number:
Kindly provide the registration number of the company as recorded at the relevant regulatory authority.
Reference Number for Corporation Tax Returns:
Please provide the reference number under which the company submits its annual corporation tax returns.
Company Background:
Describe the background of the company, including its industry, purpose, and key operations.
Innovative Solution:
Describe the innovative solution or product developed by the company, emphasizing its unique features and value proposition.
Research and Development (R&D) Tax Relief:
Please provide details about any research and development (R&D) tax relief or incentives that the company has applied for or utilized, including the applicable reference or eligibility criteria.
Please provide the information in separate sections as described above."""

for chunk in chunks:
    sentences = ' '.join(list(chunk))
    concatenated_string = ", ".join([str(item) for item in summary_responses]) + sentences  # prepend the summary for each chunk

    prompt = f"""Summarize the Transcription including the key points: incorporation date, Registration Number, Reference Number for Corporation Tax Returns, Company Background,
    Innovative Solution and Accounting period.
    The Company Background and Innovative Solution should be summarized with 200 words out of a total of a maximum of 300 words \nTranscription: {concatenated_string}\n\ntl;dr:"""

    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        temperature=0.3, # The temperature parameter controls the randomness attribute of the response, represented as a range from 0 to 1. A lower value of temperature indicates that the API will respond with the first thing that the model sees; a higher value indicates the model evaluates possible responses which could fit into the context before outputting out the result.
        max_tokens=1024,
        top_p=1, # Top P parameter controls how many random results the model should consider for model completion, as suggested by the temperature parameter, thus determining the scope of randomness. Top P’s range is from 0 to 1. A lower value imposes limitation on creativity, while a higher value expands its horizons for creativity.
        frequency_penalty=0,
        presence_penalty=1
    )

    response_text = response["choices"][0]["text"]
    summary_responses.append(response_text)

full_summary_q1 = "".join(response_text)

print("full summary for q1:")
print(full_summary_q1)



In [ ]:
# Code block for generate BERT score for question 1
# Include the candidate reference form the manually created output report
cands_q1 = '''*(From the manually created output report, I have included the section corresponding to the question 1 for comparing the semantic similarity. I have removed this due to data privacy concerns from Grantify.)'''

F1_q1, F1_mean_q1 = generate_bert_score(cands_q1, full_summary_q1)

print('F1_q1, Mean F1 q1:',F1_q1, F1_mean_q1)

F1 Score: tensor([0.5500, 0.4950, 0.5089, 0.5206, 0.5180, 0.4900, 0.5156, 0.6791])
F1 Mean Score: tensor(0.5345)
F1_q1, Mean F1 q1: tensor([0.6000, 0.5950, 0.6089, 0.6206, 0.6180, 0.5900, 0.6156, 0.5791]) tensor(0.5345)


In [ ]:
# Function to tokenize paragraohs to sentences-used by DeBERTa and RoBERTa
def paragraph_to_sentences(paragraph):
    sentences = nltk.sent_tokenize(paragraph)
    return sentences
# Converting candidate and reference paragraphs to list of sentences using the above mentioned function 'paragraph_to_sentences'
candidate_list = paragraph_to_sentences(cands_q1)
reference_list = paragraph_to_sentences(full_summary_q1)

In [ ]:
# Code block for generate DeBERTa score

similarity_matrix_deberta = compute_similarity_matrix_deberta(candidate_list, reference_list)

# Print similarity matrix
for i, candidate in enumerate(candidate_list):
    for j, reference in enumerate(reference_list):
        similarity_score = similarity_matrix_deberta[i][j]
        print(f"Similarity between '{candidate}' and '{reference}': {similarity_score:.4f}")


In [ ]:
# Code block for generate RoBERTa score

similarity_matrix_roberta = compute_similarity_matrix_roberta(candidate_list, reference_list)

# Print similarity matrix
for row in similarity_matrix_roberta:
  candidate, reference, similarity_score = row
  print(f"Candidate: {candidate}")
  print(f"Reference: {reference}")
  print(f"Similarity score: {similarity_score:.4f}")
  print()

In [ ]:
# Code block for text summarization for question 1 - added few alterations or enhancements to the earlier prompt for testing the efficiency
summary_responses = []

question = '''PROMPT:
- You are a super knowledgeable grant applicant. Write a paragraph and include only the important points to ANSWER the QUESTION (mentioned under <question> tag), quantify as much as possible.
- Try to use CONTEXT (mentioned under <context> tag), PLATFORM GUIDANCE (mentioned under <guidance> tag), REVIEWER DIRECTIONS (mentioned under <reviewer_directions> tag)
and CLIENT CONTEXT (mentioned under <client_context> tag) data while answer the below QUESTION if possible:
- Make sure to include information in CLIENT CONTEXT to the ANSWER if the CLIENT CONTEXT is not empty
- Ensure the answer follows the APA style guide
- Remove any detailed references from the end of ANSWER (if any)
- Desired format: {answer text here}
- Follow this set of INSTRUCTIONS (mentioned under <instructions> tag to generate the ANSWER
- FINAL STEP: remove all <ANSWER> and </ANSWER> tags from the answer text

<instructions>
Plan your answer before generating.  It is important to generate an answer as close as possible to the TARGET WORD COUNT, which you can find in parentheses after the QUESTION
(example: 8-A2: What are the social impacts of your project? (100 words), TARGET WORD COUNT is 100.).  To help with meeting the target word count goal, use the following algorithm:
- Plan the components of your answer.
- For each component, assign it a percentage weight of the total.  For example, if one topic or sentence accounts for half the important content in the answer, assign it a weight of 50%.
- Generate each component of the answer, matching word counts to proportionate weight.  As in the example above, for a 100-word answer, if the first topic is assigned a weight of 50%,
try to generate 1 or 2 sentences that total 50 words.
</instructions>

<question>What is the  incorporation date, Registration Number, Reference Number for Corporation Tax Returns, Company Background,
    Innovative Solution and Accounting period? (300 words), TARGET WORD COUNT is 300.</question>

<guidance>
In the first paragraph, focus on the introduction of your business: provide an overview of the project or industry of your business and introduce the main problem, challenge,
or opportunity faced by the relevant stakeholders.
In the second paragraph, focus on providing the background and Context: Provide relevant background information about the industry, technology, or concept being addressed.
Highlight any existing limitations or gaps in knowledge, technology, or solutions. Also, provide a comparison with Existing Solutions or Competitors: Discuss the limitations or shortcomings
of current solutions, technologies, or competitors. Highlight the unique features, approaches, or advantages of the solution or company being focused on.
In the third paragaraph, describe a unique Solution or Approach: Describe the innovative solution, technology, or approach being developed or implemented. Explain how this solution differs
from existing ones and addresses the identified problem or challenge. Also, provide the Progress and Challenges: Discuss the progress made by the company or project in implementing the solution.
Highlight any specific challenges or uncertainties faced in the process.
Furthermore, also provide the Developmental Requirements or Research Efforts: Explain the specific requirements or research efforts undertaken to develop or refine the solution.
Highlight the importance of various components, tests, or investigations conducted.
In the fourth paragaraph, provide Potential Impact or Significance: Discuss the potential impact, benefits, or significance of the solution or approach.
Highlight the value it brings to the industry, market, or target audience.
</guidance>

'''

for chunk in chunks:
    sentences = ' '.join(list(chunk))
    concatenated_string = ", ".join([str(item) for item in summary_responses]) + sentences # prepend the summary for each chunk

    prompt = f"""Summarize the Transcription according to the prompt mentioned in the question: {question}  \nTranscription: {concatenated_string}\n\ntl;dr:"""

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3, # The temperature controls the randomness of the response, represented as a range from 0 to 1. A lower value of temperature means the API will respond with the first thing that the model sees; a higher value means the model evaluates possible responses that could fit into the context before spitting out the result.
        max_tokens=1024,
        top_p=1, # Top P controls how many random results the model should consider for completion, as suggested by the temperature dial, thus determining the scope of randomness. Top P’s range is from 0 to 1. A lower value limits creativity, while a higher value expands its horizons.
        frequency_penalty=0,
        presence_penalty=1
    )

    response_text = response.choices[0].message.content
    summary_responses.append(response_text)

full_summary_q1 = "".join(response_text)

print("full summary q1")
print(full_summary_q1)

In [ ]:
# Generate BERT score for question 1
# Include the candidate reference form the manually created output report
cands_q1 = '''*(From the manually created output report, I have included the section corresponding to the question 1 for comparing the semantic similarity. I have removed this due to data privacy concerns from Grantify.)'''

F1_q1, F1_mean_q1 = generate_bert_score(cands_q1, full_summary_q1)

print('F1_q1, Mean F1 q1:',F1_q1, F1_mean_q1)

You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
F1 Score: tensor([0.6000, 0.5050, 0.6389, 0.6000, 0.5280, 0.5900, 0.5206, 0.5182])
F1 Mean Score: tensor(0.5625)
F1_q1, Mean F1 q1: tensor([0.6000, 0.5050, 0.6389, 0.6000, 0.5280, 0.5900, 0.5206, 0.5182]) tensor(0.5625)


In [ ]:
# Code block for generate DeBERTa score for q1

similarity_matrix_deberta = compute_similarity_matrix_deberta(cands_q1, full_summary_q1)

# Print similarity matrix
for i, candidate in enumerate(candidate_list):
    for j, reference in enumerate(reference_list):
        similarity_score = similarity_matrix_deberta[i][j]
        print(f"Similarity score: {similarity_score:.4f}")



In [ ]:
# Code block for generate RoBERTa score for q1

similarity_matrix_roberta = compute_similarity_matrix_roberta(cands_q1, full_summary_q1)

# Print similarity matrix
for row in similarity_matrix_roberta:
  candidate, reference, similarity_score = row
  print(f"Similarity score: {similarity_score:.4f}")
  print()

In [ ]:
# Code block for take user input to modify the answer generated for questoin 1 above
user_input_q1 = input("Please enter some text: ")
print("You entered:", user_input_q1)


In [ ]:
# Code block for modifying the summary generated using the user input
summary_responses = []

question = '''PROMPT:

- You are a super knowledgeable grant applicant. Write a paragraph and include only the important points to ANSWER the QUESTION (mentioned under <question> tag), quantify as much as possible.
- Try to use CONTEXT (mentioned under <context> tag), PLATFORM GUIDANCE (mentioned under <guidance> tag), REVIEWER DIRECTIONS (mentioned under <reviewer_directions> tag) and CLIENT CONTEXT (mentioned under <client_context> tag) data while answer the below QUESTION if possible:
- Make sure to include information in CLIENT CONTEXT to the ANSWER if the CLIENT CONTEXT is not empty
- Ensure the answer follows the APA style guide
- Remove any detailed references from the end of ANSWER (if any)
- Desired format: {answer text here}
- Follow this set of INSTRUCTIONS (mentioned under <instructions> tag to generate the ANSWER
- FINAL STEP: remove all <ANSWER> and </ANSWER> tags from the answer text

<instructions>
Plan your answer before generating.  It is important to generate an answer as close as possible to the TARGET WORD COUNT, which you can find in parentheses after the QUESTION (example: 8-A2: What are the social impacts of your project? (100 words), TARGET WORD COUNT is 100.).  To help with meeting the target word count goal, use the following algorithm:
- Plan the components of your answer.
- For each component, assign it a percentage weight of the total.  For example, if one topic or sentence accounts for half the important content in the answer, assign it a weight of 50%.
- Generate each component of the answer, matching word counts to proportionate weight.  As in the example above, for a 100-word answer, if the first topic is assigned a weight of 50%, try to generate 1 or 2 sentences that total 50 words.
</instructions>

<context>
When formulating your response, incorporate specific keywords associated with the company's industry, such as relevant knowledge and capability. These keywords should enhance the quality and accuracy of the generated answer.
</context>

<question>What is the  incorporation date, Registration Number, Reference Number for Corporation Tax Returns, Company Background,
    Innovative Solution and Accounting period? (300 words), TARGET WORD COUNT is 300.</question>

<guidance>In the first paragraph, focus on the introduction of your business: provide an overview of the project or industry of your business and introduce the main problem, challenge, or opportunity faced by the relevant stakeholders.
In the second paragraph, focus on providing the background and Context: Provide relevant background information about the industry, technology, or concept being addressed.
Highlight any existing limitations or gaps in knowledge, technology, or solutions. Also, provide a comparison with Existing Solutions or Competitors: Discuss the limitations or shortcomings of current solutions, technologies, or competitors.
Highlight the unique features, approaches, or advantages of the solution or company being focused on. In the third paragaraph, describe a unique Solution or Approach: Describe the innovative solution, technology, or approach being developed or implemented.
Explain how this solution differs from existing ones and addresses the identified problem or challenge. Also, provide the Progress and Challenges: Discuss the progress made by the company or project in implementing the solution.
Highlight any specific challenges or uncertainties faced in the process. Furthermore, also provide the Developmental Requirements or Research Efforts: Explain the specific requirements or research efforts undertaken to develop or refine the solution.
Highlight the importance of various components, tests, or investigations conducted.  In the fourth paragaraph, provide Potential Impact or Significance: Discuss the potential impact, benefits, or significance of the solution or approach.
Highlight the value it brings to the industry, market, or target audience.
</guidance>
'''

modified_summary_q1 = apply_hmrc_guidelines(full_summary_q1, user_input_q1)
print("Modified Summary:", modified_summary_q1)



In [ ]:
#Calculate BERT score with modified summary for question 1
# Include the candidate reference form the manually created output report for the corresponding section
print("Modified summary for q1")
cands_q1 = '''*(From the manually created output report, I have included the section corresponding to the question 1 for comparing the semantic similarity. I have removed this due to data privacy concerns from Grantify.)'''

F1_q1, F1_mean_q1 = generate_bert_score(cands_q1, modified_summary_q1)

print('F1_q1, Mean F1 q1:',F1_q1, F1_mean_q1)


Modified summary for q1
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
F1 Score: tensor([0.6500, 0.5950, 0.6089, 0.6206, 0.6180, 0.5900, 0.6156, 0.5791])
F1 Mean Score: tensor(0.6096)
F1_q1, Mean F1 q1: tensor([0.6000, 0.5950, 0.6089, 0.6206, 0.6180, 0.5900, 0.6156, 0.5791]) tensor(0.6096)



In [ ]:
# Code block for generate DeBERTa score for summary modified with user input for q1

similarity_matrix_deberta = compute_similarity_matrix_deberta(cands_q1, modified_summary_q1)

# Print similarity matrix
for i, candidate in enumerate(candidate_list):
    for j, reference in enumerate(reference_list):
        similarity_score = similarity_matrix_deberta[i][j]
        print(f"Similarity score: {similarity_score:.4f}")



In [ ]:

# Code block for generate RoBERTa score for summary modified with user input for q1

similarity_matrix_roberta = compute_similarity_matrix_roberta(cands_q1, modified_summary_q1)

# Print similarity matrix
for row in similarity_matrix_roberta:
  candidate, reference, similarity_score = row
  print(f"Similarity score: {similarity_score:.4f}")
  print()

In [ ]:
# Code block for text summarization for question 2
summary_responses = []

question = f'''PROMPT:As an exceptionally knowledgeable grant applicant, your task is to craft a paragraph that focuses solely on the essential points required to ANSWER the QUESTION
provided under the <question> tag. Quantify the information as much as possible. Utilize the CONTEXT (found under <context>), PLATFORM GUIDANCE (found under <guidance>),
REVIEWER DIRECTIONS (found under <reviewer_directions>), and CLIENT CONTEXT (found under <client_context>) data, if applicable, when constructing your response.
If the CLIENT CONTEXT is provided, ensure that the answer incorporates relevant information from it. Additionally, ensure that the answer adheres to the APA style guide.
Please follow the INSTRUCTIONS outlined under the <instructions> tag to generate your ANSWER.

<instructions>
To generate an effective response, it is crucial to plan your answer before generating it. Strive to create a well-structured answer that closely aligns with the TARGET WORD COUNT
indicated in parentheses after the QUESTION (e.g., 8-A2: What are the social impacts of your project? (100 words), TARGET WORD COUNT is 100). To assist in meeting the target word count,
utilize the following algorithm:
Identify the essential components of your answer.
Assign a percentage weight to each component based on its significance. For example, if one topic or sentence contributes to half of the essential content, assign it a weight of 50%.
Generate each component of your answer, ensuring that the word counts correspond to their proportionate weight. For instance, if the first topic carries a weight of 50% in a 100-word answer,
aim to generate 1 or 2 sentences totaling approximately 50 words.
</instructions>

<question>
What is the baseline of relevant knowledge and capability of the company? (300 words) [TARGET WORD COUNT: 300]
</question>

<context>
When formulating your response, incorporate specific keywords associated with the company's industry, such as relevant knowledge and capability.
These keywords should enhance the quality and accuracy of the generated answer.
</context>

<guidance>
In the first paragraph, provide Introduction: Provide an overview of your business, including the project or industry it operates in.
Introduce the primary problem, challenge, or opportunity faced by the relevant stakeholders.

In the second paragraph, provide Background and Context: Provide pertinent background information about the industry, technology, or concept being addressed.
Highlight any existing limitations, knowledge gaps, or shortcomings in current solutions, technologies, or competitors.
Discuss the unique features, approaches, or advantages offered by your solution or company.

In the third paragraph, provide Unique Solution or Approach: Describe the innovative solution, technology, or approach being developed or implemented.
Explain how it differentiates from existing solutions and addresses the identified problem or challenge.
Discuss the progress made in implementing the solution, including any specific challenges or uncertainties faced.
Additionally, explain the developmental requirements or research efforts undertaken to develop or refine the solution,
emphasizing the importance of various components, tests, or investigations conducted.

In the fourth paragraph, provide Potential Impact or Significance: Discuss the potential impact, benefits, or significance of the solution or approach.
Highlight the value it brings to the industry, market, or target audience.
</guidance>
'''

for chunk in chunks:
    sentences = ' '.join(list(chunk))
    concatenated_string = ", ".join([str(item) for item in summary_responses]) + sentences # prepend the summary for each chunk

    prompt = f"""Summarize the Transcription according to the prompt mentioned in the question: {question}  \nTranscription: {concatenated_string}\n\ntl;dr:"""

    # response = openai.Completion.create(
    #     engine="text-davinci-003",
    #     prompt=prompt,
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3, # The temperature controls the randomness of the response, represented as a range from 0 to 1. A lower value of temperature means the API will respond with the first thing that the model sees; a higher value means the model evaluates possible responses that could fit into the context before spitting out the result.
        max_tokens=1024,
        top_p=1, # Top P controls how many random results the model should consider for completion, as suggested by the temperature dial, thus determining the scope of randomness. Top P’s range is from 0 to 1. A lower value limits creativity, while a higher value expands its horizons.
        frequency_penalty=0,
        presence_penalty=1
    )

    response_text = response.choices[0].message.content # response["choices"][0]["text"]
    summary_responses.append(response_text)

full_summary_q2 = "".join(response_text)

print("full summary q2")
print(full_summary_q2)



In [ ]:
# Generate BERT score for question 2
# Include the candidate reference form the manually created output report for the corresponding section

cands_q2 = '''*(From the manually created output report, I have included the section corresponding to the question 2 for comparing the semantic similarity. I have removed this due to data privacy concerns from Grantify.)'''

F1_q2, F1_mean_q2 = generate_bert_score(cands_q2, full_summary_q2)

print('F1_q2, Mean F1 q2:',F1_q2, F1_mean_q2)

You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
F1 Score: tensor([0.2050, 0.0516, 0.2039, 0.1536, 0.1068, 0.5035, 0.2086, 0.3031])
F1 Mean Score: tensor(0.2168)
F1_q2, Mean F1 q2: tensor([0.2050, 0.0516, 0.2039, 0.1536, 0.1068, 0.5035, 0.2086, 0.3031]) tensor(0.2168)


In [ ]:
# Code block for generate DeBERTa score for q2

similarity_matrix_deberta = compute_similarity_matrix_deberta(cands_q2, full_summary_q2)

# Print similarity matrix
for i, candidate in enumerate(candidate_list):
    for j, reference in enumerate(reference_list):
        similarity_score = similarity_matrix_deberta[i][j]
        print(f"Similarity score: {similarity_score:.4f}")


In [ ]:
# Code block for generate RoBERTa score for q2

similarity_matrix_roberta = compute_similarity_matrix_roberta(cands_q2, full_summary_q2)

# Print similarity matrix
for row in similarity_matrix_roberta:
  candidate, reference, similarity_score = row
  print(f"Similarity score: {similarity_score:.4f}")
  print()

In [ ]:
# Code block for take user input to modify the answer generated for question 2 above
user_input_q2 = input("Please enter some text: ")
print("You entered:", user_input_q2)


In [ ]:
# Code block for modifying the summary generated using the user input
summary_responses = []

question = f'''PROMPT:

- You are a super knowledgeable grant applicant. Write a paragraph and include only the important points to ANSWER the QUESTION (mentioned under <question> tag), quantify as much as possible.
- Try to use CONTEXT (mentioned under <context> tag), PLATFORM GUIDANCE (mentioned under <guidance> tag), REVIEWER DIRECTIONS (mentioned under <reviewer_directions> tag) and CLIENT CONTEXT (mentioned under <client_context> tag) data while answer the below QUESTION if possible:
- Make sure to include information in CLIENT CONTEXT to the ANSWER if the CLIENT CONTEXT is not empty
- Ensure the answer follows the APA style guide

- Follow this set of INSTRUCTIONS (mentioned under <instructions> tag to generate the ANSWER


<instructions>
Plan your answer before generating.  It is important to generate an answer as close as possible to the TARGET WORD COUNT, which you can find in parentheses after the QUESTION (example: 8-A2: What are the social impacts of your project? (100 words), TARGET WORD COUNT is 100.).  To help with meeting the target word count goal, use the following algorithm:
- Plan the components of your answer.
- For each component, assign it a percentage weight of the total.  For example, if one topic or sentence accounts for half the important content in the answer, assign it a weight of 50%.
- Generate each component of the answer, matching word counts to proportionate weight.  As in the example above, for a 100-word answer, if the first topic is assigned a weight of 50%, try to generate 1 or 2 sentences that total 50 words.
</instructions>

<question>What is the baseline of relevant knowledge and capability of the company? (300 words), TARGET WORD COUNT is 300.</question>

<guidance>In the first paragraph, focus on the introduction of your business: provide an overview of the project or industry of your business and introduce the main problem, challenge, or opportunity faced by the relevant stakeholders.
In the second paragraph, focus on providing the background and Context: Provide relevant background information about the industry, technology, or concept being addressed.
Highlight any existing limitations or gaps in knowledge, technology, or solutions. Also, provide a comparison with Existing Solutions or Competitors: Discuss the limitations or shortcomings of current solutions, technologies, or competitors.
Highlight the unique features, approaches, or advantages of the solution or company being focused on. In the third paragaraph, describe a unique Solution or Approach: Describe the innovative solution, technology, or approach being developed or implemented.
Explain how this solution differs from existing ones and addresses the identified problem or challenge. Also, provide the Progress and Challenges: Discuss the progress made by the company or project in implementing the solution.
Highlight any specific challenges or uncertainties faced in the process. Furthermore, also provide the Developmental Requirements or Research Efforts: Explain the specific requirements or research efforts undertaken to develop or refine the solution.
Highlight the importance of various components, tests, or investigations conducted.  In the fourth paragaraph, provide Potential Impact or Significance: Discuss the potential impact, benefits, or significance of the solution or approach.
Highlight the value it brings to the industry, market, or target audience.
</guidance>

'''

modified_summary_q2 = apply_hmrc_guidelines(full_summary_q2, user_input_q2)
print("Modified Summary:", modified_summary_q2)



In [ ]:
# Generate BERT score for modified summary - question 2
# Include the candidate reference form the manually created output report

cands_q2 = '''*(From the manually created output report, I have included the section corresponding to the question 2 for comparing the semantic similarity. I have removed this due to data privacy concerns from Grantify.)'''

F1_q2, F1_mean_q2 = generate_bert_score(cands_q2, modified_summary_q2)

print('F1_q2, Mean F1 q2:',F1_q2, F1_mean_q2)

You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
F1 Score: tensor([0.2050, 0.0516, 0.2539, 0.1536, 0.1068, 0.5035, 0.3086, 0.4031])
F1 Mean Score: tensor(0.2607)
F1_q2, Mean F1 q2: tensor([0.2050, 0.0516, 0.2539, 0.1536, 0.1068, 0.5035, 0.3086, 0.4031]) tensor(0.2607)


In [ ]:
# Code block for generate DeBERTa score for summary modified with user input for q2

similarity_matrix_deberta = compute_similarity_matrix_deberta(cands_q2, modified_summary_q2)

# Print similarity matrix
for i, candidate in enumerate(candidate_list):
    for j, reference in enumerate(reference_list):
        similarity_score = similarity_matrix_deberta[i][j]
        print(f"Similarity score: {similarity_score:.4f}")

In [ ]:
# Code block for generate RoBERTa score for summary modified with user input for q2

similarity_matrix_roberta = compute_similarity_matrix_roberta(cands_q2, modified_summary_q2)

# Print similarity matrix
for row in similarity_matrix_roberta:
  candidate, reference, similarity_score = row
  print(f"Similarity score: {similarity_score:.4f}")
  print()

In [ ]:
# Code block for text summarization for question 3 - What is the scientific or technological advancement sought by the company under consideration?
summary_responses = []

question = f'''PROMPT:
- You are a super knowledgeable grant applicant. Write a paragraph and include only the important points to ANSWER the QUESTION (mentioned under <question> tag), quantify as much as possible.
- Try to use CONTEXT (mentioned under <context> tag), PLATFORM GUIDANCE (mentioned under <guidance> tag), REVIEWER DIRECTIONS (mentioned under <reviewer_directions> tag) and
CLIENT CONTEXT (mentioned under <client_context> tag) data while answer the below QUESTION if possible:
- Make sure to include information in CLIENT CONTEXT to the ANSWER if the CLIENT CONTEXT is not empty
- Ensure the answer follows the APA style guide
- Follow this set of INSTRUCTIONS (mentioned under <instructions> tag to generate the ANSWER
<context>
-Please make a conscious effort to include specific key words associated with 'Smart city solutions' and 'IoT Integration' in your responses. These key words should be relevant to the topic
being discussed and help enhance the quality and accuracy of the generated answer.
</context>
<instructions>
Plan your answer before generating.  It is important to generate an answer as close as possible to the TARGET WORD COUNT, which you can find in parentheses after the QUESTION
(example: 8-A2: What are the social impacts of your project? (100 words), TARGET WORD COUNT is 100.).  To help with meeting the target word count goal, use the following algorithm:
- Plan the components of your answer.
- For each component, assign it a percentage weight of the total.  For example, if one topic or sentence accounts for half the important content in the answer, assign it a weight of 50%.
- Generate each component of the answer, matching word counts to proportionate weight.  As in the example above, for a 100-word answer, if the first topic is assigned a weight of 50%,
try to generate 1 or 2 sentences that total 50 words.
</instructions>
<question>What is the scientific or technological advancement sought by the company under consideration? (300 words), TARGET WORD COUNT is 300.</question>
<guidance>
In the first paragraph, introduce the Prototype Design and Development of the company. Emphasize specific projects aimed at advancing science and technology in various fields.
Highlight the research conducted and the resulting designs for prototypes, emphasizing key features intended for future commercial products.
In the second paragraph, focus on the practical aspects and efficient methods employed in the projects. Discuss the assembly of components and the development of efficient methods,
emphasizing the importance of safe assembly and maintenance practices, even with limited facilities.

In the third paragraph, describe the performance characterization of the systems and the development of additional components to complete them.
Highlight the experiments conducted to characterize the electrical and mechanical performance, as well as the design and construction of critical bespoke components and novel, cost-effective solutions.

In the fourth paragraph, provide insights into the testing phase of the projects. Mention the simulation of flow conditions and the limited performance measurements due to budgetary constraints.
Highlight the validation of design decisions and the insights gained for future improvements. Explain how the projects qualify as significant advancements through scientific or
technological changes, aligning with relevant guidelines and regulations.

In the fifth paragraph, discuss various projects aimed at advancing technology in specific industries. Highlight the development of unique software platforms and their potential benefits.
Focus on enhancing predictions and data management, addressing challenges related to user adoption and managing data for store owners.
Additionally, elaborate on the development of advanced data infrastructure.

In the sixth paragraph, emphasize improving data architecture methodology and revising software to mitigate systems uncertainty.
Highlight the systematic approach of iterative testing and continuous modifications. Discuss the planning, analysis, and testing of software for delivering a reliable and feasible solution.
Finally, mention the iterative development of prototype hardware to overcome technical uncertainties, underscoring the importance of refining and improving the hardware to support the
advanced data infrastructure.
</guidance>

'''

for chunk in chunks:
    sentences = ' '.join(list(chunk))
    concatenated_string = ", ".join([str(item) for item in summary_responses]) + sentences # prepend the summary for each chunk

    prompt = f"""Summarize the Transcription according to the prompt mentioned in the question: {question}  \nTranscription: {concatenated_string}\n\ntl;dr:"""

    response = openai.ChatCompletion.create(
        # openai.Completion.create(
        # engine="text-davinci-003",
        # prompt=prompt,
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3, # The temperature controls the randomness of the response, represented as a range from 0 to 1. A lower value of temperature means the API will respond with the first thing that the model sees; a higher value means the model evaluates possible responses that could fit into the context before spitting out the result.
        max_tokens=1024,
        top_p=1, # Top P controls how many random results the model should consider for completion, as suggested by the temperature dial, thus determining the scope of randomness. Top P’s range is from 0 to 1. A lower value limits creativity, while a higher value expands its horizons.
        frequency_penalty=0,
        presence_penalty=1
    )

    response_text = response.choices[0].message.content #response["choices"][0]["text"]
    summary_responses.append(response_text)

full_summary_q3 = "".join(response_text)

print("full summary q3")
print(full_summary_q3)



In [ ]:
# Generate BERT score for question 3
# Include the candidate reference form the manually created output report
cands_q3 = '''*(From the manually created output report, I have included the section corresponding to the question 3 for comparing the semantic similarity. I have removed this due to data privacy concerns from Grantify.)'''

F1_q3, F1_mean_q3 = generate_bert_score(cands_q3, full_summary_q3)

print('F1_q3, Mean F1 q3:',F1_q3, F1_mean_q3)


You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
F1 Score: tensor([0.2050, 0.0416, 0.1039, 0.3576, 0.4006, 0.2042, 0.5119, 0.4312])
F1 Mean Score: tensor(0.2820)
F1_q3, Mean F1 q3: tensor([0.2050, 0.0416, 0.1039, 0.3576, 0.4006, 0.2042, 0.5119, 0.4312]) tensor(0.2820)


In [ ]:
# Code block for generate DeBERTa score for q3

similarity_matrix_deberta = compute_similarity_matrix_deberta(cands_q3, full_summary_q3)

# Print similarity matrix
for i, candidate in enumerate(candidate_list):
    for j, reference in enumerate(reference_list):
        similarity_score = similarity_matrix_deberta[i][j]
        print(f"Similarity Score: {similarity_score:.4f}")

In [ ]:
# Code block for generate RoBERTa score q3

similarity_matrix_roberta = compute_similarity_matrix_roberta(cands_q3, full_summary_q3)

# Print similarity matrix
for row in similarity_matrix_roberta:
  candidate, reference, similarity_score = row
  print(f"Similarity score: {similarity_score:.4f}")
  print()

In [ ]:
# Code block for modifying the summary generated using the HMRC guidelines pertaining to the question 3, i.e., advancements in science and technology
summary_responses = []

question = f'''PROMPT:

- You are a super knowledgeable grant applicant. Write a paragraph and include only the important points to ANSWER the QUESTION (mentioned under <question> tag), quantify as much as possible.
- Try to use CONTEXT (mentioned under <context> tag), PLATFORM GUIDANCE (mentioned under <guidance> tag), REVIEWER DIRECTIONS (mentioned under <reviewer_directions> tag) and CLIENT CONTEXT (mentioned under <client_context> tag) data while answer the below QUESTION if possible:
- Make sure to include information in CLIENT CONTEXT to the ANSWER if the CLIENT CONTEXT is not empty
- Ensure the answer follows the APA style guide

- Follow this set of INSTRUCTIONS (mentioned under <instructions> tag to generate the ANSWER
<context>
-Please make a conscious effort to include specific key words associated with 'Smart city solutions' and 'IoT Integration' in your responses. These key words should be relevant to the topic being discussed and help enhance the quality and accuracy of the generated answer.

</context>

<instructions>
Plan your answer before generating.  It is important to generate an answer as close as possible to the TARGET WORD COUNT, which you can find in parentheses after the QUESTION (example: 8-A2: What are the social impacts of your project? (100 words), TARGET WORD COUNT is 100.).
To help with meeting the target word count goal, use the following algorithm:
- Plan the components of your answer.
- For each component, assign it a percentage weight of the total.  For example, if one topic or sentence accounts for half the important content in the answer, assign it a weight of 50%.
- Generate each component of the answer, matching word counts to proportionate weight.  As in the example above, for a 100-word answer, if the first topic is assigned a weight of 50%, try to generate 1 or 2 sentences that total 50 words.
</instructions>

<question>What is the scientific or technological advancement sought by the company under consideration? (300 words), TARGET WORD COUNT is 300.</question>
'''

hmrc_reference="""ADVANCE IN SCIENCE OR TECHNOLOGY:
1. An advance in science or technology means an advance in
overall knowledge or capability in a field of science or
technology (not a company’s own state of knowledge or
capability alone). This includes the adaptation of knowledge
or capability from another field of science or technology in
order to make such an advance where this adaptation was
not readily deducible.
2. An advance in science or technology may have tangible
consequences (such as a new or more efficient cleaning
product, or a process which generates less waste) or more
intangible outcomes (new knowledge or cost improvements,
for example).
3. A process, material, device, product, service or source of
knowledge does not become an advance in science or
technology simply because science or technology is used in
its creation. Work which uses science or technology but
which does not advance scientific or technological capability
as a whole is not an advance in science or technology.
4. A project which seeks to, for example,
(a) extend overall knowledge or capability in a field of science
or technology; or
(b) create a process, material, device, product or service
which incorporates or represents an increase in overall
knowledge or capability in a field of science or
technology; or
(c) make an appreciable improvement to an existing
process, material, device, product or service through
scientific or technological changes; or
(d) use science or technology to duplicate the effect of an
existing process, material, device, product or service in a
new or appreciably improved way (e.g. a product which
has exactly the same performance characteristics as
existing models, but is built in a fundamentally different
manner)
will therefore be R&D.
5. Even if the advance in science or technology sought by a
project is not achieved or not fully realised, R&D still takes
place.
6. If a particular advance in science or technology has already
been made or attempted but details are not readily available
(for example, if it is a trade secret), work to achieve such an
advance can still be an advance in science or technology.
7. However, the routine analysis, copying or adaptation of an
existing product, process, service or material, will not be an
advance in science or technology."""

modified_summary_q3 = apply_hmrc_guidelines(full_summary_q3, hmrc_reference)
print("Modified Summary:", modified_summary_q3)



In [ ]:
# Generate BERT score for modified summary - question 3
# Include the candidate reference form the manually created output report
print("HMRC Modified summary for q3")
cands_q3 = '''*(From the manually created output report, I have included the section corresponding to the question 3 for comparing the semantic similarity. I have removed this due to data privacy concerns from Grantify.)'''

F1_q3, F1_mean_q3 = generate_bert_score(cands_q3, modified_summary_q3)

print('F1_q3, Mean F1 q3:',F1_q3, F1_mean_q3)

HMRC Modified summary for q3
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
F1 Score: tensor([0.3050, 0.0516, 0.2039, 0.4536, 0.5068, 0.1035, 0.5586, 0.4031])
F1 Mean Score: tensor(0.3231)
F1_q3, Mean F1 q3: tensor([0.3050, 0.0516, 0.2039, 0.4536, 0.5068, 0.1035, 0.5586, 0.4031]) tensor(0.3231)


In [ ]:
# Code block for generate DeBERTa score for summary modified with HMRC guidelines corresponding to q3

similarity_matrix_deberta = compute_similarity_matrix_deberta(cands_q3, modified_summary_q3)

# Print similarity matrix
for i, candidate in enumerate(candidate_list):
    for j, reference in enumerate(reference_list):
        similarity_score = similarity_matrix_deberta[i][j]
        print(f"Similarity Score: {similarity_score:.4f}")

In [ ]:
# Code block for generate RoBERTa score for summary modified with HMRC guidelines corresponding to q3

similarity_matrix_roberta = compute_similarity_matrix_roberta(cands_q3, modified_summary_q3)

# Print similarity matrix
for row in similarity_matrix_roberta:
  candidate, reference, similarity_score = row
  print(f"Similarity score: {similarity_score:.4f}")
  print()

In [ ]:
# Code block for text summarization for question 4 - scientific or technological uncertainties of the project under consideration
summary_responses = []
question = f'''PROMPT:
As a highly knowledgeable grant applicant, your task is to compose a paragraph that focuses on answering the QUESTION provided below by including only the important points.
Whenever possible, quantify the information. Please utilize relevant data from CONTEXT, PLATFORM GUIDANCE, REVIEWER DIRECTIONS, and CLIENT CONTEXT to enrich your response.

If CLIENT CONTEXT is available, ensure that you incorporate pertinent information into the answer.

Additionally, ensure that your answer adheres to the APA style guide.

To generate the most effective response, please follow these INSTRUCTIONS:

- Prioritize planning your answer before generating it. Aim to generate an answer that closely aligns with the TARGET WORD COUNT indicated in parentheses after the QUESTION
(e.g., 8-A2: What are the social impacts of your project? (100 words), TARGET WORD COUNT is 100).
- Break down your answer into components and assign each component a percentage weight relative to the total content. For instance, if one topic or sentence constitutes half of the important content,
assign it a weight of 50%.
- Generate each component while matching the word counts to their proportionate weights. For example, if the first topic has a weight of 50% in a 100-word answer,
aim to generate 1 or 2 sentences totaling 50 words for that topic.
<question>What are the scientific or technological uncertainties of the project under consideration? (300 words), TARGET WORD COUNT is 300.</question>
<context>
-Please make a conscious effort to include specific key words associated with 'Smart city solutions' and 'IoT Integration' in your responses.
These key words should be relevant to the topic being discussed and help enhance the quality and accuracy of the generated answer.

</context>
<guidance>
In the first paragraph, introduce the scientific or technological uncertainties of the project and highlight the scale of the endeavor. Explain the specific challenges and uncertainties involved,
while referencing the application of general principles and the project's magnitude.
In the second paragraph, discuss a specific technical challenge and its impact on the project. Provide a detailed description of the issue and elucidate how it affects the project.
Explain any requirements or variations in behavior or design arising from this challenge.

In the third paragraph, address another technical challenge and its implications on manufacturing. Highlight the impact of this challenge on the manufacturing or production process.
Emphasize the crucial areas or components affected by the challenge.

In the fourth paragraph, explore a distinct technical challenge and describe the expected differences in performance or behavior.
Discuss the need for different strategies or requirements to overcome this challenge.

In the fifth paragraph, focus on an additional technical challenge and the design of context-specific solutions.
Explain the facilities or environment related to this challenge and emphasize the importance of designing solutions tailored to the specific context.

</guidance>

'''

for chunk in chunks:
    sentences = ' '.join(list(chunk))
    concatenated_string = ", ".join([str(item) for item in summary_responses]) + sentences # prepend the summary for each chunk

    prompt = f"""Summarize the Transcription according to the prompt mentioned in the question: {question}  \nTranscription: {concatenated_string}\n\ntl;dr:"""

    # response = openai.Completion.create(
    #     engine="text-davinci-003",
    #     prompt=prompt,
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3, # The temperature controls the randomness of the response, represented as a range from 0 to 1. A lower value of temperature means the API will respond with the first thing that the model sees; a higher value means the model evaluates possible responses that could fit into the context before spitting out the result.
        max_tokens=1024,
        top_p=1, # Top P controls how many random results the model should consider for completion, as suggested by the temperature dial, thus determining the scope of randomness. Top P’s range is from 0 to 1. A lower value limits creativity, while a higher value expands its horizons.
        frequency_penalty=0,
        presence_penalty=1
    )

    response_text = response.choices[0].message.content #response["choices"][0]["text"]
    summary_responses.append(response_text)

full_summary_q4 = "".join(response_text)

print("full summary q4")
print(full_summary_q4)



In [ ]:
# Generate BERT score for question 4
# Include the candidate reference form the manually created output report
cands_q4 = '''*(From the manually created output report, I have included the section corresponding to the question 4 for comparing the semantic similarity. I have removed this due to data privacy concerns from Grantify.)'''

F1_q4, F1_mean_q4 = generate_bert_score(cands_q4, full_summary_q4)

print('F1_q4, Mean F1 q4:',F1_q4, F1_mean_q4)


You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
F1 Score: tensor([0.2050, 0.1529, 0.1382, 0.1082, 0.0967, 0.2061, 0.0916, 0.3571])
F1 Mean Score: tensor(0.1696)
F1_q4, Mean F1 q4: tensor([0.2050, 0.1529, 0.1382, 0.1082, 0.0967, 0.2061, 0.0916, 0.3571]) tensor(0.1696)


In [ ]:
# Code block for generate DeBERTa score for q4

similarity_matrix_deberta = compute_similarity_matrix_deberta(cands_q4, full_summary_q4)

# Print similarity matrix
for i, candidate in enumerate(candidate_list):
    for j, reference in enumerate(reference_list):
        similarity_score = similarity_matrix_deberta[i][j]
        print(f"Similarity Score: {similarity_score:.4f}")

In [ ]:
# Code block for generate RoBERTa score for q4

similarity_matrix_roberta = compute_similarity_matrix_roberta(cands_q4, full_summary_q4)

# Print similarity matrix
for row in similarity_matrix_roberta:
  candidate, reference, similarity_score = row
  print(f"Similarity score: {similarity_score:.4f}")
  print()

In [ ]:
# Code block for modifying the summary generated using the HMRC guidelines pertaining to question 4 - scientific or technological uncertainties of the project under consideration
summary_responses = []

question = f'''PROMPT:
As a highly knowledgeable grant applicant, your task is to compose a paragraph that focuses on answering the QUESTION provided below by including only the important points. Whenever possible, quantify the information. Please utilize relevant data from CONTEXT, PLATFORM GUIDANCE, REVIEWER DIRECTIONS, and CLIENT CONTEXT to enrich your response.

If CLIENT CONTEXT is available, ensure that you incorporate pertinent information into the answer.

Additionally, ensure that your answer adheres to the APA style guide.

To generate the most effective response, please follow these INSTRUCTIONS:

- Prioritize planning your answer before generating it. Aim to generate an answer that closely aligns with the TARGET WORD COUNT indicated in parentheses after the QUESTION (e.g., 8-A2: What are the social impacts of your project? (100 words), TARGET WORD COUNT is 100).
- Break down your answer into components and assign each component a percentage weight relative to the total content. For instance, if one topic or sentence constitutes half of the important content, assign it a weight of 50%.
- Generate each component while matching the word counts to their proportionate weights. For example, if the first topic has a weight of 50% in a 100-word answer, aim to generate 1 or 2 sentences totaling 50 words for that topic.
<question>What are the scientific or technological uncertainties of the project under consideration? (300 words), TARGET WORD COUNT is 300.</question>
<context>
-Please make a conscious effort to include specific key words associated with 'Smart city solutions' and 'IoT Integration' in your responses. These key words should be relevant to the topic being discussed and help enhance the quality and accuracy of the generated answer.

</context>
<guidance>
In the first paragraph, introduce the scientific or technological uncertainties of the project and highlight the scale of the endeavor. Explain the specific challenges and uncertainties involved, while referencing the application of general principles and the project's magnitude.
In the second paragraph, discuss a specific technical challenge and its impact on the project. Provide a detailed description of the issue and elucidate how it affects the project. Explain any requirements or variations in behavior or design arising from this challenge.

In the third paragraph, address another technical challenge and its implications on manufacturing. Highlight the impact of this challenge on the manufacturing or production process. Emphasize the crucial areas or components affected by the challenge.

In the fourth paragraph, explore a distinct technical challenge and describe the expected differences in performance or behavior. Discuss the need for different strategies or requirements to overcome this challenge.

In the fifth paragraph, focus on an additional technical challenge and the design of context-specific solutions. Explain the facilities or environment related to this challenge and emphasize the importance of designing solutions tailored to the specific context.

</guidance>
'''

hmrc_reference="""SCIENTIFIC OR TECHNOLOGICAL UNCERTAINTY:
1) Scientific or technological uncertainty exists when
knowledge of whether something is scientifically possible or
technologically feasible, or how to achieve it in practice, is not
readily available or deducible by a competent professional
working in the field. This includes system uncertainty.
Scientific or technological uncertainty will often arise from
turning something that has already been established as
scientifically feasible into a cost-effective, reliable and
reproducible process, material, device, product or service.
2) Uncertainties that can readily be resolved by a competent
professional working in the field are not scientific or
technological uncertainties. Similarly, improvements,
optimisations and fine-tuning which do not materially affect
the underlying science or technology do not constitute work
to resolve scientific or technological uncertainty."""

modified_summary_q4 = apply_hmrc_guidelines(full_summary_q4, hmrc_reference)
print("Modified Summary:", modified_summary_q4)



In [ ]:
# Generate BERT score for modified summary - question 4
# Include the candidate reference form the manually created output report
print("Modified summary for q4")
cands_q4 = '''*(From the manually created output report, I have included the section corresponding to the question 4 for comparing the semantic similarity. I have removed this due to data privacy concerns from Grantify.)'''

F1_q4, F1_mean_q4 = generate_bert_score(cands_q4, modified_summary_q4)

print('F1_q4, Mean F1 q4:',F1_q4, F1_mean_q4)

Modified summary for q4
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
F1 Score: tensor([0.2350, 0.2516, 0.2439, 0.1736, 0.0268, 0.3035, 0.1286, 0.4531])
F1 Mean Score: tensor(0.2520)
F1_q4, Mean F1 q4: tensor([0.2350, 0.2516, 0.2439, 0.1736, 0.0268, 0.3035, 0.1286, 0.4531]) tensor(0.2520)


In [ ]:
# Code block for generate DeBERTa score for summary modified with HMRC guidelines corresponding to q4

similarity_matrix_deberta = compute_similarity_matrix_deberta(cands_q4, modified_summary_q4)

# Print similarity matrix
for i, candidate in enumerate(candidate_list):
    for j, reference in enumerate(reference_list):
        similarity_score = similarity_matrix_deberta[i][j]
        print(f"Similarity Score: {similarity_score:.4f}")

In [ ]:
# Code block for generate RoBERTa score for summary modified with HMRC guidelines corresponding to q4

similarity_matrix_roberta = compute_similarity_matrix_roberta(cands_q4, modified_summary_q4)

# Print similarity matrix
for row in similarity_matrix_roberta:
  candidate, reference, similarity_score = row
  print(f"Similarity score: {similarity_score:.4f}")
  print()

In [ ]:
# Code block for text summarization for question 5 - How the uncertainties were overcome by the company under consideration?
summary_responses = []

question = f'''PROMPT:
- You are a super knowledgeable grant applicant. Write a paragraph and include only the important points to ANSWER the QUESTION (mentioned under <question> tag), quantify as much as possible.
- Try to use CONTEXT (mentioned under <context> tag), PLATFORM GUIDANCE (mentioned under <guidance> tag), REVIEWER DIRECTIONS (mentioned under <reviewer_directions> tag) and
CLIENT CONTEXT (mentioned under <client_context> tag) data while answer the below QUESTION if possible:
- Make sure to include information in CLIENT CONTEXT to the ANSWER if the CLIENT CONTEXT is not empty
- Ensure the answer follows the APA style guide
- Follow this set of INSTRUCTIONS (mentioned under <instructions> tag to generate the ANSWER
<instructions>
Plan your answer before generating.  It is important to generate an answer as close as possible to the TARGET WORD COUNT, which you can find in parentheses after the QUESTION
(example: 8-A2: What are the social impacts of your project? (100 words), TARGET WORD COUNT is 100.).  To help with meeting the target word count goal, use the following algorithm:
- Plan the components of your answer.
- For each component, assign it a percentage weight of the total.  For example, if one topic or sentence accounts for half the important content in the answer, assign it a weight of 50%.
- Generate each component of the answer, matching word counts to proportionate weight.  As in the example above, for a 100-word answer, if the first topic is assigned a weight of 50%,
try to generate 1 or 2 sentences that total 50 words.
</instructions>
<question>How the uncertainties were overcome by the company under consideration?  (450 words), TARGET WORD COUNT is 450.</question>
<context>
-Please make a conscious effort to include specific key words associated with 'Smart city solutions' and 'IoT Integration' in your responses.
These key words should be relevant to the topic being discussed and help enhance the quality and accuracy of the generated answer.
</context>
<guidance>
In the first paragraph, focus on the Introduction: Briefly introduce the uncertainties faced by the subject of the answer.
Mention the importance of identifying and resolving these uncertainties. Highlight the allocation of resources and commitment to resolving the uncertainties.
In the second paragraph, focus on Uncertainty Exploration and Identification:Discuss the specific uncertainties encountered and their impact.
Explain how research and professional expertise were used to identify these uncertainties. Emphasize the need for further investigation and resolution.

In the third paragaraph, describe a Resolution Strategies and Challenges: Outline the strategies and approaches taken to overcome the uncertainties.
Describe the challenges and obstacles faced during the resolution process. Highlight any partnerships or collaborations formed to address the uncertainties.

In the fourth paragaraph, provide Specific Uncertainties and Solutions:
Dedicate paragraphs to each major uncertainty discussed in the answer. Explain the nature of the uncertainty and its implications.
Detail the specific solutions or approaches employed to resolve the uncertainty. Mention any iterations, modifications, or trial-and-error processes involved.

In the fifth paragaraph, provide Discussion of Ongoing Work and Future Opportunities:
Acknowledge that not all uncertainties have been fully resolved. Discuss ongoing work beyond the current timeframe or accounting period.
Highlight the future opportunities for improvement, scalability, or research.
</guidance>

'''

for chunk in chunks:
    sentences = ' '.join(list(chunk))
    concatenated_string = ", ".join([str(item) for item in summary_responses]) + sentences # prepend the summary for each chunk

    prompt = f"""Summarize the Transcription according to the prompt mentioned in the question: {question}  \nTranscription: {concatenated_string}\n\ntl;dr:"""

    response = openai.ChatCompletion.create(
        # openai.Completion.create(
        # engine="text-davinci-003",
        # prompt=prompt,
        model="gpt-4", #"gpt-3.5-turbo"
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3, # The temperature controls the randomness of the response, represented as a range from 0 to 1. A lower value of temperature means the API will respond with the first thing that the model sees; a higher value means the model evaluates possible responses that could fit into the context before spitting out the result.
        max_tokens=1024,
        top_p=1, # Top P controls how many random results the model should consider for completion, as suggested by the temperature dial, thus determining the scope of randomness. Top P’s range is from 0 to 1. A lower value limits creativity, while a higher value expands its horizons.
        frequency_penalty=0,
        presence_penalty=1
    )

    response_text = response.choices[0].message.content #response["choices"][0]["text"]
    summary_responses.append(response_text)

full_summary_q5 = "".join(response_text)

print("full summary q5")
print(full_summary_q5)



In [ ]:
# Generate BERT score for question 5
# Include the candidate reference form the manually created output report
cands_q5 = '''*(From the manually created output report, I have included the section corresponding to the question 5 for comparing the semantic similarity. I have removed this due to data privacy concerns from Grantify.)'''

F1_q5, F1_mean_q5 = generate_bert_score(cands_q5, full_summary_q5)

print('F1_q5, Mean F1 q5:',F1_q5, F1_mean_q5)


You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
F1 Score: tensor([0.2100, 0.0628, 0.2981, 0.1491, 0.4811, 0.3817, 0.4193, 0.4819])
F1 Mean Score: tensor(0.3105)
F1_q5, Mean F1 q5: tensor([0.2100, 0.0628, 0.2981, 0.1491, 0.4811, 0.3817, 0.4193, 0.4819]) tensor(0.3105)


In [ ]:
# Code block for generate DeBERTa score for q5

similarity_matrix_deberta = compute_similarity_matrix_deberta(cands_q5, full_summary_q5)

# Print similarity matrix
for i, candidate in enumerate(candidate_list):
    for j, reference in enumerate(reference_list):
        similarity_score = similarity_matrix_deberta[i][j]
        print(f"Similarity Score: {similarity_score:.4f}")

In [ ]:
# Code block for generate RoBERTa score for q5

similarity_matrix_roberta = compute_similarity_matrix_roberta(cands_q5, full_summary_q5)

# Print similarity matrix
for row in similarity_matrix_roberta:
  candidate, reference, similarity_score = row
  print(f"Similarity score: {similarity_score:.4f}")
  print()

In [ ]:
# Code block for modifying the summary generated using the HMRC guidelines pertaining to question 5 - SCIENTIFIC OR TECHNOLOGICAL UNCERTAINTY
summary_responses = []
question = f'''PROMPT:

- You are a super knowledgeable grant applicant. Write a paragraph and include only the important points to ANSWER the QUESTION (mentioned under <question> tag), quantify as much as possible.
- Try to use CONTEXT (mentioned under <context> tag), PLATFORM GUIDANCE (mentioned under <guidance> tag), REVIEWER DIRECTIONS (mentioned under <reviewer_directions> tag) and CLIENT CONTEXT (mentioned under <client_context> tag) data while answer the below QUESTION if possible:
- Make sure to include information in CLIENT CONTEXT to the ANSWER if the CLIENT CONTEXT is not empty
- Ensure the answer follows the APA style guide

- Follow this set of INSTRUCTIONS (mentioned under <instructions> tag to generate the ANSWER


<instructions>
Plan your answer before generating.  It is important to generate an answer as close as possible to the TARGET WORD COUNT, which you can find in parentheses after the QUESTION (example: 8-A2: What are the social impacts of your project? (100 words), TARGET WORD COUNT is 100.).  To help with meeting the target word count goal, use the following algorithm:
- Plan the components of your answer.
- For each component, assign it a percentage weight of the total.  For example, if one topic or sentence accounts for half the important content in the answer, assign it a weight of 50%.
- Generate each component of the answer, matching word counts to proportionate weight.  As in the example above, for a 100-word answer, if the first topic is assigned a weight of 50%, try to generate 1 or 2 sentences that total 50 words.
</instructions>

<question>How the uncertainties were overcome by the company under consideration?  (450 words), TARGET WORD COUNT is 450.</question>
<context>
-Please make a conscious effort to include specific key words associated with 'Smart city solutions' and 'IoT Integration' in your responses. These key words should be relevant to the topic being discussed and help enhance the quality and accuracy of the generated answer.

</context>
<guidance>In the first paragraph, focus on the Introduction: Briefly introduce the uncertainties faced by the subject of the answer.
Mention the importance of identifying and resolving these uncertainties.
Highlight the allocation of resources and commitment to resolving the uncertainties.


In the second paragraph, focus on Uncertainty Exploration and Identification:Discuss the specific uncertainties encountered and their impact.
Explain how research and professional expertise were used to identify these uncertainties.
Emphasize the need for further investigation and resolution.


In the third paragaraph, describe a Resolution Strategies and Challenges:
Outline the strategies and approaches taken to overcome the uncertainties.
Describe the challenges and obstacles faced during the resolution process.
Highlight any partnerships or collaborations formed to address the uncertainties.

In the fourth paragaraph, provide Specific Uncertainties and Solutions:
Dedicate paragraphs to each major uncertainty discussed in the answer.
Explain the nature of the uncertainty and its implications.
Detail the specific solutions or approaches employed to resolve the uncertainty.
Mention any iterations, modifications, or trial-and-error processes involved.


In the fifth paragaraph, provide Discussion of Ongoing Work and Future Opportunities:
Acknowledge that not all uncertainties have been fully resolved.
Discuss ongoing work beyond the current timeframe or accounting period.
Highlight the future opportunities for improvement, scalability, or research.
</guidance>
'''

hmrc_reference="""SCIENTIFIC OR TECHNOLOGICAL UNCERTAINTY:
1) Scientific or technological uncertainty exists when
knowledge of whether something is scientifically possible or
technologically feasible, or how to achieve it in practice, is not
readily available or deducible by a competent professional
working in the field. This includes system uncertainty.
Scientific or technological uncertainty will often arise from
turning something that has already been established as
scientifically feasible into a cost-effective, reliable and
reproducible process, material, device, product or service.
2) Uncertainties that can readily be resolved by a competent
professional working in the field are not scientific or
technological uncertainties. Similarly, improvements,
optimisations and fine-tuning which do not materially affect
the underlying science or technology do not constitute work
to resolve scientific or technological uncertainty."""


modified_summary_q5 = apply_hmrc_guidelines(full_summary_q5, hmrc_reference)
print("Modified Summary:", modified_summary_q5)



In [ ]:
# Generate BERT score for modified summary - question 5
# Include the candidate reference form the manually created output report

cands_q5 = '''*(From the manually created output report, I have included the section corresponding to the question 5 for comparing the semantic similarity. I have removed this due to data privacy concerns from Grantify.)'''

F1_q5, F1_mean_q5 = generate_bert_score(cands_q5, modified_summary_q5)

print('F1_q5, Mean F1 q5:',F1_q5, F1_mean_q5)


You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
F1 Score: tensor([0.2350, 0.0416, 0.3539, 0.1536, 0.4568, 0.4035, 0.4886, 0.5031])
F1 Mean Score: tensor(0.3293)
F1_q5, Mean F1 q5: tensor([0.2350, 0.0416, 0.3539, 0.1536, 0.4568, 0.4035, 0.4886, 0.5031]) tensor(0.3293)


In [ ]:
# Code block for generate DeBERTa score for summary modified by corresponding HMRC guidelines for q5

similarity_matrix_deberta = compute_similarity_matrix_deberta(cands_q5, modified_summary_q5)

# Print similarity matrix
for i, candidate in enumerate(candidate_list):
    for j, reference in enumerate(reference_list):
        similarity_score = similarity_matrix_deberta[i][j]
        print(f"Similarity Score: {similarity_score:.4f}")

In [ ]:
# Code block for generate RoBERTa score for summary modified by corresponding HMRC guidelines for q5

similarity_matrix_roberta = compute_similarity_matrix_roberta(cands_q5, modified_summary_q5)

# Print similarity matrix
for row in similarity_matrix_roberta:
  candidate, reference, similarity_score = row
  print(f"Similarity score: {similarity_score:.4f}")
  print()

In [ ]:
# check BERT installation
import bert_score
bert_score.__version__

'0.3.12'